# 8 聚类方法

给定数据 $x_i\in\mathbb R^p\ (i=1,\dotsc,n)$, 将其 (按照位置关系等) 分为 $k$ 个类, 即为聚类 (clustering).



## 度量

### 距离

可以按照距离关系将 $x_i\ (i=1,\dotsc,n)$ 分类, 距离可以不止是欧氏距离:

#### 闵可夫斯基距离

闵可夫斯基距离 (Minkowski distance): $(m\geqslant 1)$
$$d_{ij} = \Vert x_i - x_j \Vert_m =\left(\sum_{k=1}|x_{ik}  - x_{jk}|^m\right)^\frac 1m$$

特别地, 当 $m = 2$ 是欧氏距离 (Euclidean distance), $m = 1$ 是曼哈顿距离 (Manhattan distance), $m = + \infty$ 是切比雪夫距离 (Chebyshev distance).


#### 马氏距离

马氏距离: $S$ 为正定矩阵,
$$d_{ij} = \Vert x_i - x_j\Vert_{S^{-1}} = \sqrt{(x_i - x_j)^TS^{-1}(x_i - x_j)}$$


### 相似度

除距离外, 可以定义两个数据的相似度, 如余弦相似度或相关系数. 不同的相似度定义往往介于 $[-1,1]$ (或 $[0,1]$) 之间.

#### 余弦相似度

余弦相似度 (cosine similarity) 是两个数据向量的夹角余弦值.

$$s_{ij} = \langle x_i, x_j\rangle = \frac{x_i^Tx_j}{\Vert x_i\Vert_2\Vert x_j\Vert_2}$$

#### 相关系数

相关系数 (correlation coefficient) 将两个数据看成两组 $d$ 个独立同分布观测并计算协方差.
$$r_{ij} = \frac{\sum_{k=1}^p (x_{ik}- \bar x_i)(x_{jk} - \bar x_j)}{\sqrt{\sum_{k=1}^p (x_{ik} - \bar x_i)^2\sum_{k=1}^p (x_{jk} - \bar x_j)^2}}$$

#### 转换

数据越相似, 相似度越大, 距离越小. 可见两个指标是的意义并不相同. 不过可以相互转换. 例如可以定义 $距离 = \sqrt{2(1 - 相似度)}$ 或 $相似度 = (1+距离)^{-1}$.

## 簇

聚类结果分好的每一个类称为簇 / 类 (cluster). 假设 $G$ 是一个簇, 定义其中心为该簇内的数据的平均值:
$$\bar x_G = \frac{1}{|G|}\sum_{x_i\in G}x_i$$

定义其直径为最远的两个数据的距离:
$$D_G = \max_{x_i,x_j\in G}d_{ij}$$

### 簇间距离

两个簇 $G_a, G_b$ 之间的距离, 亦称类连接 (linkage), 可用如下定义衡量:

#### 最短距离

最短距离又称为单连接 (single linkage), 是两个簇之间最近的两个数据的距离:
$$D_{ab} = \min_{x_i\in G_a,\ x_j\in G_b} d_{ij}$$

#### 最长距离

最长距离又称为完全连接 (complete linkage),  是两个簇之间最远的两个数据的距离:
$$D_{ab} =  \max_{x_i\in G_a,\ x_j\in G_b} d_{ij}$$

#### 中心距离

即两个簇的簇中心的距离
$$D_{ab} = d(\bar x_a, \bar x_b)$$

#### 平均距离

即两个簇数据点的距离的平均值
$$D_{ab} = \frac{1}{|G_a||G_b|}\sum_{x_i\in G_a}\sum_{x_j\in G_b}d_{ij}$$

## 层次聚类

每次选择最近两个类融合为一个新类.